Полное число частиц:
\begin{equation}
    n = \frac{N}{V} = \frac{g m^{3 / 2}}{\sqrt{2} \pi^{2} \hbar^{3}} \int_{0}^{\infty} \frac{\sqrt{\varepsilon} d \varepsilon}{e^{(\varepsilon-\mu) / T} + 1}
    \label{eq-N}
\end{equation}

Энергия:
\begin{equation}
    E=\int_{0}^{\infty} \varepsilon d N_{\varepsilon}=\frac{g V m^{3 / 2}}{\sqrt{2} \pi^{2} \hbar^{3}} \int_{0}^{\infty} \frac{\varepsilon^{3 / 2} d \varepsilon}{e^{(\varepsilon-\mu) / T} + 1}
    \label{eq-E}
\end{equation}

Для заданных $T, N/V$ надо найти $E$, а затем и $P$ по формуле:
\begin{equation}
    PV = (2/3) E
\end{equation}

\begin{equation}
    \int_{0}^{\infty} \frac{\varepsilon^{n/2} d \varepsilon}{e^{\varepsilon / T-\mu / T} + 1} = \left|
    \begin{array}
        1\varepsilon / T = t; & d\varepsilon = T dt \\
        \varepsilon = t T;   & \varepsilon^{n/2} = (t T)^{n/2} \\
        \mu / T = x & \\
    \end{array}
    \right| = \int_{0}^{\infty} \frac{t^{n / 2} T^{n/2} T dt}{e^{t-x} + 1} = 
    T^{(n+2)/2} \int_{0}^{\infty} \frac{t^{n / 2} dt}{e^{t-x} + 1} = 
\end{equation}
\begin{equation}
    = T^{(n+2)/2} \, \Gamma((n+2)/2) F_{n/2} (x) = T^{(n+2)/2} \, \Gamma\left(\frac{n+2}{2}\right) F_{n/2} \left(\frac{\mu}{T}\right)
\end{equation}
где $n = 1$ или $2$, а $T = kT'$, где $T'$ выражена в СИ (Кельвин). 

Для числа частиц имеем:
\begin{equation}
    \frac{E}{V} \, \frac{\sqrt{2} \pi^{2} \hbar^{3}}{g m^{3 / 2}} - T^{3 / 2} \Gamma(3/2) F_{1 / 2}\left(\frac{\mu}{T}\right) = 0
\end{equation}

\begin{equation}
    F_{1 / 2}\left(\frac{\mu}{T}\right) = \frac{E}{V} \, \frac{\sqrt{2} \pi^{2} \hbar^{3}}{g m^{3 / 2}} \frac{1}{ T^{3 / 2} \Gamma(3/2) }
\end{equation}

Отсюда получаем $\mu$, а затем подставляем в выражение для $E$:
\begin{equation}
    E = \frac{g V m^{3 / 2}}{\sqrt{2} \pi^{2} \hbar^{3}} \int_{0}^{\infty} \frac{\varepsilon^{3 / 2} d \varepsilon}{e^{(\varepsilon-\mu) / T}+1} =
    \frac{g V m^{3 / 2}}{\sqrt{2} \pi^{2} \hbar^{3}} T^{5 / 2} \, \Gamma(5/2) \, F_{3 / 2}\left(\frac{\mu}{T}\right)
\end{equation}

\begin{equation}
    P = (2/3) E/V = \frac{\sqrt{2}g m^{3 / 2}}{3 \pi^{2} \hbar^{3}} T^{5 / 2} \, \Gamma(5/2) \, F_{3 / 2}\left(\frac{\mu}{T}\right)
\end{equation}

In [1]:
from scipy.constants import electron_mass, hbar, pi, k, N_A, electron_volt
from scipy.special import gamma
from fdint import *
import numpy as np
from scipy.optimize import bisect, brentq

In [2]:
def get_mu_from_N_div_V(n, T, g=2, m=electron_mass):
    # T is in Kelvins!
    T = T*k
    F_right = n * (np.sqrt(2) * pi**2 * hbar**3 /(g*m**(3/2)) ) / (T**(3/2) * gamma(3/2))
    # We find minimum order 10**n for optimization
    fermi_int = lambda mu: fdk(k=0.5, phi=mu)
    max_order = 0
    # TODO: хим.потенциал может быть отрицательным!
    # Подбираем порядок
    while fermi_int(10**max_order) < F_right:
        max_order += 1
    # Если нужно пойти в отрицательные степени
    if max_order == 0 and fermi_int(10**max_order) >= F_right:
        while fermi_int(10 ** max_order) >= F_right:
            max_order -= 1
    max_order += 1
    min_order = max_order - 2
    tmp = lambda mu: fermi_int(mu) - F_right
    mu = brentq(lambda mu: fermi_int(mu) - F_right, 10**min_order, 10**max_order)
    return mu

In [66]:
fdk(0, 8) - np.log(1+np.exp(8))

0.0

In [3]:
def get_E_div_V_from_mu(mu, T, g=2, m=electron_mass):
    # T is in Kelvins!
    T = T*k
    return g*m**(3/2) / (np.sqrt(2) * pi**2 * hbar**3) * T**(5/2)*gamma(5/2)*fdk(k=1.5, phi=mu/T)

In [4]:
def get_mu_E(N_div_V, T):
    mu_div_kT_calc = get_mu_from_N_div_V(N_div_V, T)
    # Хим.потенциал
    mu_calc = mu_div_kT_calc * k * T
    # Полная энергия
    E_calc = get_E_div_V_from_mu(mu_calc, T)
    return mu_calc, E_calc

In [18]:
# Попробуем эту алхимию применить!
# Возьмем алюминий, rho = 2.7 г/см^3
N_div_V = (2.7/27 * 10**6) * N_A * 3
T = 10**-5
mu, E = get_mu_E(N_div_V, T)
print('T = %s K: \nmu = %s эВ, E/V = %s Дж/м^3' % (T, mu/electron_volt, E))
print('P = %s ГПа' % (E*2/3/10**9))

T = 1e-05 K: 
mu = 12.630535060140446 эВ, E/V = 329038572535.665 Дж/м^3
P = 219.35904835711 ГПа


In [6]:
# Медь
N_div_V = (8.92/63.5* 10**6) * N_A * 3
mu, E = get_mu_E(N_div_V, T)
print('T = %s K: \nmu = %s эВ, E/V = %s Дж/м^3' % (T, mu/electron_volt, E))
print('P = %s ГПа' % (E*2/3/10**9))

T = 1e-05 K: 
mu = 15.842212512949502 эВ, E/V = 579738346554.2914 Дж/м^3
P = 386.49223103619426 ГПа


### Сравним с асимптотическими формулами

\begin{equation}
    \frac{E}{V}=\frac{3\left(3 \pi^{2}\right)^{2 / 3}}{10} \frac{\hbar^{2}}{m}\left(\frac{N}{V}\right)^{5 / 3}
\end{equation}

\begin{equation}
    P=\frac{\left(3 \pi^{2}\right)^{2 / 3}}{5} \frac{\hbar^{2}}{m}\left(\frac{N}{V}\right)^{5 / 3}
\end{equation}

In [7]:
def get_E_div_V_asymp(n):
    return 3*(3*pi)**(2/3)/10 * hbar**2/electron_mass * n**(5/3)
def get_P_asymp(n):
    return (3*pi**2)**(2/3)/5 * hbar**2/electron_mass * n**(5/3)

In [13]:
E_div_V_asymp = get_E_div_V_asymp(N_div_V)
print('E/V = %s Дж/м^3' % (E_div_V_asymp))

E/V = 166240746531.1324 Дж/м^3


In [ ]:
print('P = %s ГПа' % (get_P_asymp(N_div_V)/10**9))